In [1]:
import pymysql
import requests
import pandas as pd
import numpy as np

### Se conecta con la BD MySQL AWS

In [2]:
con = pymysql.connect(
      host='database-proyecto-final.cpjrl88bhdul.us-east-1.rds.amazonaws.com',
      database='databaseuci',
      user='admin',
      password='datascience'
)

### Genera un df con el archivo `csv` "D_CPT"

In [3]:
df = pd.read_csv('C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\D_CPT.csv', encoding='UTF-8', delimiter=',', decimal='.')
df.head(3)

,row_id,category,sectionrange,sectionheader,subsectionrange,subsectionheader,codesuffix,mincodeinsubsection,maxcodeinsubsection
0,1,1,99201-99499,Evaluation and management,99201-99216,Office/other outpatient services,NaN,99201,99216
1,2,1,99201-99499,Evaluation and management,99217-99220,Hospital observation services,NaN,99217,99220
2,3,1,99201-99499,Evaluation and management,99221-99239,Hospital inpatient services,NaN,99221,99239


In [4]:
df = df.replace(np.nan, 0)

### Inserta los registros del df "D_CPT" en la BD

In [5]:
sql_insert = ''' INSERT 
                   INTO  D_CPT
                         (row_id, category, sectionrange, sectionheader, subsectionrange, subsectionheader, codesuffix, 
                          mincodeinsubsection, maxcodeinsubsection)
                 VALUES  (%s, %s, %s, %s, %s, %s, %s, %s, %s) '''
num_reng = 0
cursor = con.cursor()
for i in range(0, len(df)):
    row = int(df.loc[i, 'row_id'])
    catg = int(df.loc[i, 'category'])
    sern = df.loc[i, 'sectionrange']
    sehd = df.loc[i, 'sectionheader']
    subr = df.loc[i, 'subsectionrange']
    subh = df.loc[i, 'subsectionheader']
    cdsf = df.loc[i, 'codesuffix']
    minc = int(df.loc[i, 'mincodeinsubsection'])
    maxc = int(df.loc[i, 'maxcodeinsubsection'])
    reg_s = (row, catg, sern, sehd, subr, subh, cdsf, minc, maxc)
    cursor.execute(sql_insert, reg_s)
    num_reng += 1
    if num_reng == 100:
        con.commit()
        num_reng = 0

### Confirma la inserción de los registros a la BD

In [6]:
con.commit()

### Cierra el cursor y la conexion a la BD

In [7]:
cursor.close()
con.close()